27.02.2018 -- Email from Steve: 

also be interesting:

    -  calculate the total snow water path (SWP, integrated SWC throughout the depth of the cloud)
$\rightarrow$ could then compare the model SWP with the retrieval SWP to see if we are in the right ball park
i.e. see if the models are producing the right amount of snow to the first order.  This also may give us further insight into how to deal with the units.   



In [1]:
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/weather_mast/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/Retrieval/')
import netCDF4
import fill_values as fv
import datetime
import calc_date as cd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import save_fig as sF
import createFolder as cF
from matplotlib.gridspec import GridSpec
import plt_ce_image as im
import read_MRR as pMRR

from scipy.integrate import simps

In [2]:
year = '2016'
month = '12'
#t = ['17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27']
t = ['20', '21', '22', '23', '24', '25', '26', '27']
tid = '00'
station = 'Haukeliseter'


### Figures 
savefig = 1
figdir = '../../Figures/Retrieval_MEPS/ensemble_spread'
cF.createFolder('%s/' %(figdir))
form = 'png'
################################################################
### MEPS
level = 'sfc'
nc_dir_sfc = '../../Data/MEPS/%s/%s_%s' %(station,level,tid)
level = 'ml'
nc_dir_ml = '../../Data/MEPS/%s/%s_%s' %(station,level,tid)
# air temperature
var_name_air_temp = 'air_temperature_ml'
air_temp_dir = '%s/%s' %(nc_dir_ml,var_name_air_temp)
# snow fall amount
var_name_snow = 'snowfall_amount_ml'
snow_dir = '%s/%s' %(nc_dir_ml,var_name_snow)
# graupel fall amount
var_name_graupel = 'graupelfall_amount_ml'
graupel_dir = '%s/%s' %(nc_dir_ml,var_name_graupel)
# atmosphere_cloud_ice_content_ml
var_name_cl_ice = 'atmosphere_cloud_ice_content_ml'
cl_ice_dir = '%s/%s' %(nc_dir_ml,var_name_cl_ice)

################################################################
### MRR
MRR_dir = '../../Data/MRR/processed_MRR'

################################################################
### Retrieval 
nc_dir_retrieval = '../../Data/Retrieved_SWC'

################################################################

In [3]:
## MRR ######
fnMRR      = dict()
time_MRR   = dict()
height_MRR = dict()
Ze         = dict()

In [4]:
## RETRIEVAL ######
fnRet      = dict()
retrieved_snowfall_amount = dict()
SWC        = dict()
SWP_ret    = dict() 
SWC_con    = dict()
time_con   = dict()
SWP_mean_ret= dict()

In [5]:
air_temp_file     = dict()
snow_file         = dict()
graupel_file      = dict()
cl_ice_file       = dict()

fn_air_temp       = dict()
fn_snow           = dict()
fn_graupel        = dict()
fn_cl_ice         = dict()

time_ml           = dict()
air_temp_ml       = dict()
pressure_ml       = dict()
snow_amount_ml    = dict()
graupel_amount_ml = dict()
cl_ice_ml         = dict()

thickness_ml      = dict()
ice_amount_ml     = dict()
time              = dict()
height_ml         = dict()
height            = dict()
density_ml        = dict()
SWP_model         = dict()



In [6]:
Rd = 287.    # gas constant for dry air [J kg^-1 K^-1]

In [7]:
for day in t:
## MRR ######  
    fnMRR[day] = netCDF4.Dataset('%s/VMRR_%s%s%s.nc' %(MRR_dir,year,month,day) ,'r')
    time_MRR[day]   = fnMRR[day].variables['time'][:]
    height_MRR[day] = fnMRR[day].variables['height'][:]

    Ze[day] = pMRR.read_and_mask(fnMRR[day],'Ze', np.nan)         # vertical Ze profile for retrieval

## RETRIEVAL ######
    fnRet[day] = netCDF4.Dataset('%s/out_swc_b6_%s%s%s.nc' %(nc_dir_retrieval, year, month, day))

    retrieved_snowfall_amount[day] = fnRet[day].variables['retrieved_snowfall_amount'][:]
    SWC[day] = fnRet[day].variables['SWC'][:]
    
    swpret = []
    swpret = simps(SWC[day][:,:], height_MRR[day], axis = 0)
    SWP_ret[day] = swpret


### hourly SWC 
mean_SWC = dict()
for day in t:
    if day == '20':
        time_con[day]= np.concatenate((time_MRR[day], time_MRR[str(int(day)+1)][:30]))
        SWC_con[day] = np.concatenate((SWC[day], SWC[str(int(day)+1)][:,:30]),axis=1)
    elif day == '27':
        time_con[day]= np.concatenate((time_MRR[str(int(day)-1)][23*60+30:], time_MRR[day]))
        SWC_con[day] = np.concatenate((SWC[str(int(day)-1)][:,23*60+30:], SWC[day]),axis=1)
    else:
        time_con[day]= np.concatenate((time_MRR[str(int(day)-1)][23*60+30:], time_MRR[day], time_MRR[str(int(day)+1)][:30]))
        SWC_con[day] = np.concatenate((SWC[str(int(day)-1)][:,23*60+30:], SWC[day], SWC[str(int(day)+1)][:,:30]),axis=1)
    
    avg60 = []
    if day == '20':
        for i in range(30,SWC_con[day].shape[1],60):
            avg60.append(np.nanmean(SWC_con[day][:,i:(i+60)],axis=1))
        mean_SWC[day] = np.transpose(avg60)
    else:
        for i in range(0,SWC_con[day].shape[1],60):
            avg60.append(np.nanmean(SWC_con[day][:,i:(i+60)],axis=1))
        mean_SWC[day] = np.transpose(avg60)

### hourly SWP
    swpret = []
    swpret = simps(mean_SWC[day][:,:], height_MRR[day], axis = 0)
    SWP_mean_ret[day] = swpret

# for plotting, only values larger than 0.01
    a,b = np.where(retrieved_snowfall_amount[day][:,:] < 0.01)
    SWC[day][a[:],b[:]]                       = np.nan
    retrieved_snowfall_amount[day][a[:],b[:]] = np.nan
    np.transpose(Ze[day])[a[:],b[:]]          = np.nan
#    mean_SWC[day][a[:],b[:]]                  = np.nan
    a1,b1 = np.where(mean_SWC[day][:,:] < 0.01)    
    mean_SWC[day][a1[:],b1[:]] = np.nan
    
    fnMRR[day].close()
    fnRet[day].close()

/Volumes/SANDISK128/Documents/Thesis/Python/Retrieval/read_MRR.py:95: RuntimeWarning: invalid value encountered in less
  mask = np.ma.getmaskarray(var[:,:])
/Volumes/SANDISK128/Documents/Thesis/Python/Retrieval/read_MRR.py:96: RuntimeWarning: invalid value encountered in less
  var = np.ma.array(var[:,:], mask = mask, fill_value = fill_value)


In [8]:
# layer for average snow water content within 200 m
nlay= 15
bot = []
top = []
for k in range(0, nlay):
    bot.append(100.+ k*200)
    top.append(bot[k]+200)

avg_SWC_ml = dict()
h_avg      = dict()
t_avg      = dict()

In [10]:
for k in range(0,np.asarray(t).shape[0]):
#for k in range(2,3):
    day = t[k]
## MEPS ######
    for ens_memb in range(0,10):
        air_temp_file[ens_memb]= '%s/%s%s%s_%s_%s.nc' %(air_temp_dir,year,month,day,tid,ens_memb)
        snow_file[ens_memb]    = '%s/%s%s%s_%s_%s.nc' %(snow_dir,year,month,day,tid,ens_memb)
        graupel_file[ens_memb] = '%s/%s%s%s_%s_%s.nc' %(graupel_dir,year,month,day,tid,ens_memb)
        cl_ice_file[ens_memb]  = '%s/%s%s%s_%s_%s.nc' %(cl_ice_dir,year,month,day,tid,ens_memb)
        
        fn_air_temp[ens_memb]  = netCDF4.Dataset(air_temp_file[ens_memb])
        fn_snow[ens_memb]      = netCDF4.Dataset(snow_file[ens_memb])
        fn_graupel[ens_memb]   = netCDF4.Dataset(graupel_file[ens_memb]) 
        fn_cl_ice[ens_memb]    = netCDF4.Dataset(cl_ice_file[ens_memb])

# Read in time to know initialisation time (plotting)    
        time_ml[ens_memb] = fn_snow[ens_memb].variables['time']
        time_ml[ens_memb] = fv.fill_nan(time_ml[ens_memb][:])
        

        ini_day = datetime.datetime.utcfromtimestamp(time_ml[0][0]).day   # day of initialisation
        hh = datetime.datetime.utcfromtimestamp(time_ml[0][0]).hour       # first hour of initialisation?
        calday, calmon = cd.get_dayname(year, month, ini_day)
        
# Read in the pressure, and the air temperature to calculate ice amount from [kg/kg] to [kg/m^3]
        air_temp_ml[ens_memb] = fn_air_temp[ens_memb].variables[var_name_air_temp]
        pressure_ml[ens_memb] = fn_snow[ens_memb].variables['pressure_ml']
# Read in the variable name which should be plotted  
        snow_amount_ml[ens_memb]    = fn_snow[ens_memb].variables[var_name_snow]
        graupel_amount_ml[ens_memb] = fn_graupel[ens_memb].variables[var_name_graupel]
        cl_ice_ml[ens_memb]         = fn_cl_ice[ens_memb].variables[var_name_cl_ice]


# create an array with time of the shape of the variable
        lead_time   = np.arange(0,snow_amount_ml[ens_memb].shape[0])
        model_level = np.arange(0,snow_amount_ml[ens_memb].shape[1])
        td,Y        = np.meshgrid(lead_time,model_level)

# substitute missing values with nan 
        pressure_ml[ens_memb]       = fv.fill_nan(pressure_ml[ens_memb][:])
        air_temp_ml[ens_memb]       = fv.fill_nan(air_temp_ml[ens_memb][:])
        snow_amount_ml[ens_memb]    = fv.fill_nan(snow_amount_ml[ens_memb][:])
        graupel_amount_ml[ens_memb] = fv.fill_nan(graupel_amount_ml[ens_memb][:])
        cl_ice_ml[ens_memb]         = fv.fill_nan(cl_ice_ml[ens_memb][:])
# get ice amount (snow+graupel)
        ice_amount_ml[ens_memb]     = snow_amount_ml[ens_memb] + graupel_amount_ml[ens_memb] + cl_ice_ml[ens_memb]

    
# layer thickness to calculate height
        thickness_ml[ens_memb]      = fn_snow[ens_memb].variables['layer_thickness']
        thickness_ml[ens_memb]      = fv.fill_nan(thickness_ml[ens_memb][:])
    
        thickness = thickness_ml[ens_memb]
        h_above = []
        h_above.append(np.zeros(thickness[:,:].shape[0]))
        th_arr = np.nansum([np.asarray(h_above)[:,0], thickness[:,0]], axis = 0)
        h_above.append(th_arr)
        for i in range(2,thickness[:,:].shape[1]):
            th_arr = np.nansum([h_above[i-1], thickness[:,i-1]], axis = 0)
            h_above.append(th_arr)
        height_ml[ens_memb] = np.transpose(h_above) 
        
        
####
# calculate density at each level
        density_ml[ens_memb] = pressure_ml[ens_memb]/(Rd*air_temp_ml[ens_memb])
              

# convert from [kg/kg] to [kg/m^3]
        ice_amount_ml[ens_memb] = (ice_amount_ml[ens_memb]*density_ml[ens_memb])

# convert the snow amount from [kg/m^2] into [g/m^2] by multiply with 1000
        ice_amount_ml[ens_memb] = (ice_amount_ml[ens_memb]*1000.)
    
    
# calculate mean of snowfall amount in each layer, every 200m
        SWC_ml = []
        h_mid = []
        for k in range(0, nlay):
            tidx,hidx = np.where(np.logical_and(height_ml[ens_memb][:,:] >= bot[k], height_ml[ens_memb][:,:] < top[k]))
# average in layer
            SWC_ml.append(np.nanmean(ice_amount_ml[ens_memb][:,hidx[:]], axis = 1))
            h_mid.append((bot[k]+top[k])/2)
        avg_SWC_ml[ens_memb] = SWC_ml      
        
        time[ens_memb]          = td
        height[ens_memb]        = height_ml[ens_memb]   
        
        hidx = []
        tidx = []
        for i in range(0,np.asarray(avg_SWC_ml[ens_memb]).shape[1]):
            hidx.append(h_mid)
        h_avg[ens_memb] = hidx
        for i in range(0,np.asarray(avg_SWC_ml[ens_memb]).shape[0]):
            tidx.append(time[ens_memb][0,:])
        t_avg[ens_memb] = tidx
        
        
## only use values below 3km
        h1, h2 = np.where(height[ens_memb][:,:] > 3000)
        ice_amount_ml[ens_memb][h1[:],h2[:]] = np.nan
        time[ens_memb][h2[:],h1[:]]          = -9999
        height[ens_memb][h1[:],h2[:]]        = np.nan
        
        
        if len(ice_amount_ml[ens_memb]) == 0:
            continue
        else:
            a2,b2 = np.where(ice_amount_ml[ens_memb][:,:] < 0.01)    ## 0.001? retrieved_snowfall_amount = iwcpsd*0.85*e-3*3600*24
            ice_amount_ml[ens_memb][a2[:],b2[:]] = np.nan
        if len(avg_SWC_ml[ens_memb]) == 0:
            continue
        else:
            a3,b3 = np.where(np.asarray(avg_SWC_ml[ens_memb]) < 0.01)
            avg_SWC_ml[ens_memb] = np.asarray(avg_SWC_ml[ens_memb])
            avg_SWC_ml[ens_memb][a3[:],b3[:]] = np.nan        

    print('%s, %s %s %s' %(calday, day, calmon, year))
    # retrieval 
    r1, r2 = np.where(mean_SWC[day] == np.nanmax(mean_SWC[day] ))
    if len(r1) == 0 or len(r2) == 0:
        print('max SWC in retrieval: NaN')
    else: 
        print('max SWC in retrieval: %.2f kg m^-3 in %.1f m; @ %.02d UTC' %(np.nanmax(mean_SWC[day]),height_MRR[day][r1],r2))
    

    # MEPS
    ml1, ml2 = np.where(np.transpose(avg_SWC_ml[0]) == np.nanmax(np.transpose(avg_SWC_ml[0])[:24,:] ))
    if len(ml1) == 0 or len(ml2) == 0:
        print('max SWC in ml       : NaN')
    else:
        print('max SWC in ml 0th EM: %.2f kg m^-3 in %.1f m; @ %.02d UTC' %(np.nanmax(np.transpose(avg_SWC_ml[0])[:24,:]), np.asarray(h_avg[0])[ml1,ml2], datetime.datetime.utcfromtimestamp(time_ml[0][ml1]).hour))
            
        
        

    val = np.zeros(shape=(10,avg_SWC_ml[0].shape[0]))
    ice_EM = []
    ice_mean = []
    ice_median = []
    ice_75 = []
    ice_25 = []
    ice_max = []
    ice_min = []
    ice_std = []
    variable = []
# calculate the ensemble mean of ice amount (all 10 ensemble member averaged)
#    for k in range(0,ice_amount_ml[0].shape[0]):
    for k in range(0,25):
#    for k in range(0,1):
        for ens_memb in range(0,10):
            val[ens_memb] = (np.transpose(avg_SWC_ml[ens_memb])[k,:])
        variable.append(val)
    ### std, mean
        ice_std.append(np.nanstd(val,axis=0,ddof=1))  
        ice_mean.append(np.nanmean(val,axis=0))

# MEPS
    ml11, ml22 = np.where((ice_mean) == np.nanmax(np.asarray(ice_mean)[:,:] ))
    if len(ml1) == 0 or len(ml2) == 0:
        print('max SWC in ml EM     : NaN')
    else:
        print('max SWC in ml EM mean: %.2f kg m^-3 in %.1f m; @ %.02d UTC' %(np.nanmax(np.asarray(ice_mean)[:,:]), np.asarray(h_avg[0])[ml11,ml22], datetime.datetime.utcfromtimestamp(time_ml[0][ml11]).hour))

        
        
# get only valuable values not nan    
        time_ml[ens_memb]       = (time_ml[ens_memb][~np.isnan(time_ml[ens_memb])])
        time[ens_memb]          = (td[:,~np.isnan(avg_SWC_ml[ens_memb]).any(axis=1)])
        height[ens_memb]        = (height_ml[ens_memb][~np.isnan(avg_SWC_ml[ens_memb]).any(axis=1),:])
        thickness_ml[ens_memb]  = (thickness_ml[ens_memb][~np.isnan(avg_SWC_ml[ens_memb]).any(axis=1),:]) 
        air_temp_ml[ens_memb]   = (air_temp_ml[ens_memb][~np.isnan(avg_SWC_ml[ens_memb]).any(axis=1),:])
        pressure_ml[ens_memb]   = (pressure_ml[ens_memb][~np.isnan(avg_SWC_ml[ens_memb]).any(axis=1),:])
        avg_SWC_ml[ens_memb]    = (ice_amount_ml[ens_memb][~np.isnan(avg_SWC_ml[ens_memb]).any(axis=1)])   
      
    
#if len(ice_std) == 0:
 #   continue 
#else:
#a4,b4 = np.where(np.asarray(ice_std) < 0.01)
#ice_std = np.asarray(ice_std)
#ice_std[a4[:],b4[:]] = np.nan 

#a5,b5 = np.where(np.asarray(ice_mean) < 0.01)
#ice_mean = np.asarray(ice_mean)
#ice_mean[a5[:],b5[:]] = np.nan 

### plot
    fig_name = '%s%s%s.%s' %(year,month,ini_day,form)

    fig = plt.figure(1, figsize=(20,7))
    ax = fig.add_subplot(111)

#    levels = np.arange(0,1.51,0.01)
    levels = np.arange(0,1.51,0.25)
    v_min=0.
    v_max=1.5
    xmax=24.
    ymax=3000.
    xticks=im.xticks2
    yticks=im.yticks1
    xlabel=im.times

    im0 = ax.contourf(np.transpose(t_avg[0])[:25,:], np.asarray(h_avg[0])[:25,:], ice_std, levels,
            cmap='jet', extend = 'max', alpha = 1.0, 
                       vmin = v_min, vmax = v_max, origin = 'lower')
#    thlevs = np.arange(0,1.51,0.1)
    levels2 = np.arange(0,1.51,0.1)
    cth = ax.contour(np.transpose(t_avg[0])[:25,:], np.asarray(h_avg[0])[:25,:], ice_mean, levels2, colors='k', 
                 linewidths=2.)
    plt.clabel(cth, fontsize = 15., inline=1, fmt = '%0.1f')

# set the limits of the plot to the limits of the data
    ax.axis([0., xmax, 0., ymax])

# labels 
    im.labels_x(ax,xticks,xlabel)
    im.labels_y(ax,yticks,im.yl1,'height [km]')
# add colorbar   
#    im.add_colorbar(fig,im0, ax, levels[::30] ,var_label='SWC')
    im.add_colorbar(fig,im0, ax, levels ,var_label='SWC')


    labels = ['ensemble mean']
    for i in range(len(labels)):
        cth.collections[i].set_label(labels[i])

    lgd = ax.legend(loc='upper left',fontsize=im.label_fs)

# title
    title = 'initalised: %s, %s %s %s %s UTC' %(calday,ini_day,calmon,year,hh)
    ax.set_title(title, fontsize=im.fontsize, color =im.date_blue )
# tight layout
    fig.tight_layout(pad=1.4,  h_pad=2.5)
    fig.subplots_adjust(top=0.94)    
    
    
    if savefig ==1:
        sF.save_figure_portrait(figdir, fig_name, form)
        print('saved: %s/%s' %(figdir, fig_name))
    else:
        plt.show()
    plt.close()
    
    for ens_memb in range(0,10):
        fn_snow[ens_memb].close()
        fn_air_temp[ens_memb].close() 
        fn_cl_ice[ens_memb].close

/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in greater_equal
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in less
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:82: RuntimeWarning: Mean of empty slice
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in greater
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:109: RuntimeWarning: invalid value encountered in less
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in less


Tuesday, 20 Dec 2016
max SWC in retrieval: NaN
max SWC in ml 0th EM: 0.04 kg m^-3 in 400.0 m; @ 10 UTC
max SWC in ml EM mean: 0.03 kg m^-3 in 400.0 m; @ 10 UTC


/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:120: RuntimeWarning: All-NaN slice encountered
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1423: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:156: RuntimeWarning: Mean of empty slice
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:169: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 1; dimension is 67 but corresponding boolean dimension is 15
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:170: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 67 but corresponding boolean dimension

saved: ../../Figures/Retrieval_MEPS/ensemble_spread/20161220.png
Wednesday, 21 Dec 2016
max SWC in retrieval: 1.08 kg m^-3 in 600.0 m; @ 16 UTC
max SWC in ml 0th EM: 2.11 kg m^-3 in 1400.0 m; @ 20 UTC
max SWC in ml EM mean: 1.24 kg m^-3 in 1400.0 m; @ 20 UTC
saved: ../../Figures/Retrieval_MEPS/ensemble_spread/20161221.png
Thursday, 22 Dec 2016
max SWC in retrieval: 1.46 kg m^-3 in 1200.0 m; @ 10 UTC
max SWC in ml 0th EM: 1.35 kg m^-3 in 1400.0 m; @ 12 UTC
max SWC in ml EM mean: 1.54 kg m^-3 in 2200.0 m; @ 14 UTC
saved: ../../Figures/Retrieval_MEPS/ensemble_spread/20161222.png
Friday, 23 Dec 2016
max SWC in retrieval: 0.91 kg m^-3 in 600.0 m; @ 23 UTC
max SWC in ml 0th EM: 0.54 kg m^-3 in 400.0 m; @ 20 UTC
max SWC in ml EM mean: 0.54 kg m^-3 in 400.0 m; @ 20 UTC
saved: ../../Figures/Retrieval_MEPS/ensemble_spread/20161223.png
Saturday, 24 Dec 2016
max SWC in retrieval: 1.39 kg m^-3 in 1000.0 m; @ 06 UTC
max SWC in ml 0th EM: 0.73 kg m^-3 in 1400.0 m; @ 17 UTC
max SWC in ml EM mean: 0.70